In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS, cross_origin

app = Flask(__name__)

CORS(app)
cors = CORS(app, resources={
        r"/*" : {
            "origins" : "*"
        }
    })


@app.route('/moderator', methods=['GET','POST'])
def moderate():
    if request.method == 'POST':
        text = request.json['text']
        from nltk.util import pr
        import pandas as pd
        import numpy as np
        from sklearn.feature_extraction.text import CountVectorizer
        from sklearn.model_selection import train_test_split
        from sklearn.tree import DecisionTreeClassifier

        data = pd.read_csv("hate.csv")
        #print(data.head())

        data["labels"] = data["class"].map({0: "Hate Speech", 1: "Offensive Language", 2: "No Hate and Offensive"})
        #print(data.head())

        data = data[["tweet", "labels"]]
        #print(data.head())

        import re
        import nltk
        stemmer = nltk.SnowballStemmer("english")
        from nltk.corpus import stopwords
        import string
        stopword=set(stopwords.words('english'))

        def clean(text):
            text = str(text).lower()
            text = re.sub('\[.*?\]', '', text)
            text = re.sub('https?://\S+|www\.\S+', '', text)
            text = re.sub('<.*?>+', '', text)
            text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
            text = re.sub('\n', '', text)
            text = re.sub('\w*\d\w*', '', text)
            text = [word for word in text.split(' ') if word not in stopword]
            text=" ".join(text)
            text = [stemmer.stem(word) for word in text.split(' ')]
            text=" ".join(text)
            return text
        data["tweet"] = data["tweet"].apply(clean)
        #print(data.head())

        x = np.array(data["tweet"])
        y = np.array(data["labels"])

        cv = CountVectorizer()
        X = cv.fit_transform(x) # Fit the Data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

        clf = DecisionTreeClassifier()
        clf.fit(X_train,y_train)
        clf.score(X_test,y_test)

        #sample = input("Enter any line : ")
        sample = text
        data = cv.transform([sample]).toarray()
        res = clf.predict(data).tolist()
        print(res)
        print(type(res))

        result = { "Result" : res[0] }

        return jsonify(result)
    
    if request.method == 'GET':
        return "<h1> Content Moderator API </h1>"

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [10/Aug/2022 21:58:23] "OPTIONS /moderator HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 21:59:47] "POST /moderator HTTP/1.1" 200 -


['Hate Speech']
<class 'list'>
